In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 100

In [2]:
data = pd.read_csv('input/train.csv')

In [3]:
data.shape

(10000, 20)

In [4]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


### train_test_split

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [7]:
train.shape, valid.shape

((7000, 20), (3000, 20))

### Prepare train, valid

In [8]:
distr_info1 = train['DistrictId'].value_counts(normalize=True).reset_index().rename(columns={'index':'DistrictId', 
                                                                               'DistrictId':'flat_qty_distr'})

In [9]:
def add_district_info1(df, distr_info1):
    df = pd.merge(df, distr_info1, on='DistrictId', how='left')
    df['flat_qty_distr'] = df['flat_qty_distr'].fillna(0.000143)
    return df

In [10]:
distr_stat_dr = train.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].mean().\
    rename(columns={'Price':'mean_price_dr'})
    
distr_stat_r = train.groupby(['Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price_r'})

mean_price = train['Price'].mean()

In [11]:
def add_stats(df, distr_stat_dr, distr_stat_r, mean_price):
    df = pd.merge(df, distr_stat_dr, on=['DistrictId', 'Rooms'], how='left')
    df = pd.merge(df, distr_stat_r, on='Rooms', how='left')
    
    df['mean_price_r'] = df['mean_price_r'].fillna(mean_price)
    df['mean_price_dr'] = df['mean_price_dr'].fillna(df['mean_price_r'])
    return df

In [12]:
def add_cat_fts(df, cat_fts=('Ecology_2', 'Ecology_3', 'Shops_2')):
    for col in cat_fts:
        df[col] = (df[col] == 'B').astype(int)
    return df

In [13]:
def fillna_healthcare_1(df):
    df['Healthcare_1'] = df['Healthcare_1'].fillna(0)
    return df

In [14]:
def prepare_data(df, distr_info1, distr_stat_dr, distr_stat_r, mean_price):
    df = add_district_info1(df, distr_info1)
    df = add_stats(df, distr_stat_dr, distr_stat_r, mean_price)
    df = add_cat_fts(df)
    df = fillna_healthcare_1(df)
    return df

In [15]:
train = prepare_data(train, distr_info1, distr_stat_dr, distr_stat_r, mean_price)

In [16]:
train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,flat_qty_distr,mean_price_dr,mean_price_r
0,14604,23,1.0,41.681380,22.796166,8.0,14,17.0,2015,0.075779,1,1,6,1437,3,0.0,0,2,1,88504.384965,0.056286,102427.030975,160134.810901
1,5621,23,3.0,163.495333,161.504222,12.0,5,3.0,1977,0.014073,1,1,2,475,0,0.0,0,0,1,207007.956663,0.056286,165911.129700,290867.452543
2,235,87,1.0,39.710131,19.538663,8.0,4,17.0,1986,0.100456,1,1,43,7227,0,0.0,1,6,0,182126.280899,0.003000,169596.630515,160134.810901
3,16258,48,3.0,96.056784,98.152802,1.0,15,1.0,2017,0.041125,1,1,46,9515,5,0.0,1,10,1,524365.550705,0.008857,382424.639356,290867.452543
4,10773,77,3.0,79.195825,44.421062,10.0,16,17.0,1984,0.298205,1,1,16,4048,3,0.0,1,3,1,322048.433990,0.004000,251751.766701,290867.452543


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 23 columns):
Id                7000 non-null int64
DistrictId        7000 non-null int64
Rooms             7000 non-null float64
Square            7000 non-null float64
LifeSquare        5514 non-null float64
KitchenSquare     7000 non-null float64
Floor             7000 non-null int64
HouseFloor        7000 non-null float64
HouseYear         7000 non-null int64
Ecology_1         7000 non-null float64
Ecology_2         7000 non-null int32
Ecology_3         7000 non-null int32
Social_1          7000 non-null int64
Social_2          7000 non-null int64
Social_3          7000 non-null int64
Healthcare_1      7000 non-null float64
Helthcare_2       7000 non-null int64
Shops_1           7000 non-null int64
Shops_2           7000 non-null int32
Price             7000 non-null float64
flat_qty_distr    7000 non-null float64
mean_price_dr     7000 non-null float64
mean_price_r      7000 non-null float

In [18]:
valid = prepare_data(valid, distr_info1, distr_stat_dr, distr_stat_r, mean_price)

### Model

In [19]:
train.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price', 'flat_qty_distr',
       'mean_price_dr', 'mean_price_r'],
      dtype='object')

In [20]:
feats = ['Rooms', 'Square', 'flat_qty_distr', 'mean_price_dr', 'Helthcare_2', 'Healthcare_1', 'HouseYear']

In [21]:
from sklearn.ensemble import RandomForestRegressor as RF

In [22]:
# ?RF

In [23]:
model = RF(n_estimators=300, max_depth=12, random_state=42, max_features=4, min_samples_leaf=2)

In [24]:
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [25]:
pred_train = model.predict(train.loc[:, feats])

In [26]:
pred_train.shape

(7000,)

In [27]:
pred_train

array([ 93557.50779231, 193197.38917049, 175801.54000418, ...,
       240602.56681924, 204675.32818292, 399193.50580112])

In [28]:
pred_valid = model.predict(valid.loc[:, feats])

In [29]:
pred_valid.shape

(3000,)

In [30]:
pred_valid

array([188078.2573157 , 337788.61879459, 217087.7267442 , ...,
       274195.46828096, 112673.01281904, 260862.25254602])

### Evaluate model

In [31]:
from sklearn.metrics import r2_score as r2

In [32]:
r2(train['Price'], pred_train)

0.8978579318090508

In [33]:
r2(valid['Price'], pred_valid)

0.683087842651711

### Test

In [34]:
# ?prepare_data

In [35]:
test = pd.read_csv('input/test.csv')

In [36]:
test = prepare_data(test, distr_info1, distr_stat_dr, distr_stat_r, mean_price)

In [37]:
test['Price'] = model.predict(test.loc[:, feats])

In [38]:
test.loc[:, ['Id', 'Price']].to_csv('AAnonymous_predictions.csv', index=None)

In [39]:
import random

In [40]:
columns = list(train.columns)
columns.remove('Price')

In [41]:
from itertools import compress

def filt(genes):
    return list(compress(train.columns, genes))

In [42]:
len(train.columns)

23

In [43]:
genes = [random.choice([True, False]) for _ in range(len(columns))]
genes

[True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True]

In [49]:
import random


class Chromosome:
    def __init__(self, generate=False):
        if generate:
            self._generate()
        self.fitness = 0
        self.done = False
        self.just_done = False
        self.mutated = False

    def _generate(self):
        self.genes = [random.choice([True, False]) for _ in range(len(columns))]

    def set_fitness(self):
        if self.done:
            self.just_done = False
        else:
            self.fitness = sum([int(b) for b in self.genes])
            self.done = self.just_done = True

    @property
    def genes_str(self):
        return [int(b) for b in self.genes].__str__()

    def __str__(self):
        just_done = '*' if self.just_done else ' '
        mutated = 'm' if self.mutated else ' '
        return f'{self.fitness:5} {self.genes_str} {just_done} {mutated}'


def cross(chro1, chro2):
    genes_len = len(chro1.genes)
    r1 = random.randint(0, genes_len - 1)
    r2 = random.randint(r1 + 1, genes_len)

    child1 = Chromosome()
    child2 = Chromosome()

    child1.genes = chro1.genes[:r1] + chro2.genes[r1:r2] + chro1.genes[r2:]
    child2.genes = chro2.genes[:r1] + chro1.genes[r1:r2] + chro2.genes[r2:]

    return child1, child2


class Ga:
    def __init__(self, size):
        self.generate(size)

    def generate(self, size):
        self.generation = [Chromosome(True) for _ in range(size)]
        self.set_fitnesses()
        self._sort()
        self.iteration = 1

    def next(self):
        end = len(self.generation)
        middle = end // 2
        gener = self.generation
        for i in range(middle, end, 2):
            parent1, parent2 = random.sample(range(middle), 2)
            gener[i], gener[i + 1] = cross(gener[parent1], gener[parent2])
        self.set_fitnesses()
        self._sort()
        self.iteration += 1

    def set_fitnesses(self):
        for chro in self.generation:
            chro.set_fitness()
            
    def _sort(self):
        self.generation = sorted(self.generation, key=lambda x: x.fitness, reverse=True)

    def __str__(self):
        ret = f'================== {self.iteration:3} ==================\n'
        OUTPUT_TOP = 10
        if len(self.generation) < OUTPUT_TOP:
            OUTPUT_TOP = len(self.generation)

        for i in range(OUTPUT_TOP):
            ret += f'{i:2})'
            ret += self.generation[i].__str__()
            ret += '\n'
        return ret


ga = Ga(100)
print(ga)

for i in range(1, 100):
    ga.next()
    print(ga)

==================   1 ==================
 0)   16 [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0] *  
 1)   16 [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1] *  
 2)   16 [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1] *  
 3)   16 [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0] *  
 4)   15 [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0] *  
 5)   15 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0] *  
 6)   15 [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1] *  
 7)   14 [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0] *  
 8)   14 [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1] *  
 9)   14 [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1] *  

==================   2 ==================
 0)   18 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1] *  
 1)   17 [1, 0, 1, 1, 0, 1, 1, 0, 1